In [1]:
# import required libraries
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

/Users/modeedna/opt/anaconda3/envs/ANLY501/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
# import required dfs
allSquads = pd.read_csv('../../data/cleanData/cleanAllSquadsV2.csv', index_col=False)
quali = pd.read_csv('../../data/cleanData/qualiClean.csv', index_col=False)
countries = pd.read_csv('../../data/cleanData/cleanSquadsPerWorldCup.csv', index_col=False)

In [18]:
# merge two out of 3 dfs
countryQuali = pd.merge(countries,quali,left_on=['Country','Year'],right_on=['Team','Year'],how='left').drop(['Team','FifaRanking','MedianCaps','MedianAge','eliteTeamPlayers'],axis=1)
# fix year column
countryQuali['Year'] = pd.to_datetime(countryQuali['Year'], format='%Y')

In [19]:
# create age column
allSquads.Year = pd.to_datetime(allSquads.Year, format='%Y')
allSquads['Date.of.birth..age.'] = pd.to_datetime(allSquads['Date.of.birth..age.'], format='%Y-%m-%d')
allSquads['Age'] = (allSquads.Year - allSquads['Date.of.birth..age.'])/np.timedelta64(1, 'Y')
# fix negative ages with mean of column
allSquads.loc[allSquads['Age']<0,'Age'] = allSquads['Age'].mean()
# group third df
groupedDF = allSquads.groupby(['Year','Country']).aggregate({'Caps': np.mean, 'eliteClub': np.sum, 'Age': np.mean}).reset_index()

In [20]:
# merge to get final dataframe
all = pd.merge(countryQuali,groupedDF,left_on=['Year','Country'],right_on=['Year','Country'],how='left')

# select desired columns
all = all[['Country','Group','Year','Host','PastGroup','DifficultGroup','RankFIFA','QualiStreak','Caps','eliteClub','Age']]

In [21]:
# export dataframe without normalization
all.to_csv('../../data/cleanData/allTables.csv')

In [22]:
# normalize data
allNorm = all
cols_to_norm = ['RankFIFA','QualiStreak','Caps','eliteClub','Age']
allNorm[cols_to_norm] = allNorm[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [25]:
# export normalized data
allNorm.to_csv('../../data/cleanData/allTablesNorm.csv')